# Convolutional Neural Network

This notebook contains tests of the CNN architecture. Results will later be copied to the main Jupyter Notebook in the root folder of the Project 1.

In [14]:
from tensorflow import keras
import os
import cv2
import numpy as np


from model.CNN import CNN

data_path = os.path.abspath('../data/cinic-10_image_classification_challenge-dataset/train')


In [2]:
train_ds = keras.preprocessing.image_dataset_from_directory(
    data_path,
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=(32, 32),
    batch_size=16
)

Found 90000 files belonging to 10 classes.
Using 72000 files for training.


In [3]:
class_names = train_ds.class_names
print(f'Classes: {class_names}')

Classes: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [4]:
cnn = CNN()
model = cnn.prepare_single_CNN_layer((32, 32, 3), len(class_names))

In [6]:
model.compile(
    optimizer='adamax',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [7]:
history = model.fit(
  train_ds,
  epochs=10
)

Epoch 1/10


4500/4500 [==============================] - 568s 126ms/step - loss: 4.3025 - accuracy: 0.3304
Epoch 2/10
4500/4500 [==============================] - 510s 113ms/step - loss: 1.4771 - accuracy: 0.4708
Epoch 3/10
4500/4500 [==============================] - 501s 111ms/step - loss: 1.0281 - accuracy: 0.6403
Epoch 4/10
4500/4500 [==============================] - 505s 112ms/step - loss: 0.6110 - accuracy: 0.7975
Epoch 5/10
4500/4500 [==============================] - 513s 114ms/step - loss: 0.3293 - accuracy: 0.8980
Epoch 6/10
4500/4500 [==============================] - 519s 115ms/step - loss: 0.1863 - accuracy: 0.9461
Epoch 7/10
4500/4500 [==============================] - 517s 115ms/step - loss: 0.1205 - accuracy: 0.9685
Epoch 8/10
4500/4500 [==============================] - 521s 116ms/step - loss: 0.0877 - accuracy: 0.9775
Epoch 9/10
4500/4500 [==============================] - 515s 114ms/step - loss: 0.0656 - accuracy: 0.9841
Epoch 10/10
4500/4500 [=====================

In [13]:
test_path = os.path.abspath('../data/cinic-10_image_classification_challenge-dataset/test')

resultFile = []

total = 0
list = os.scandir(test_path)
# print(len(list))
print(sum(1 for _ in list))
# for testPicture in list:
#     image_to_predict = cv2.imread(testPicture.path, cv2.IMREAD_COLOR)
#     img_to_predict = np.expand_dims(cv2.resize(image_to_predict, (32, 32)), axis=0)
    
#     #prediction
#     res = np.argmax(model.predict(img_to_predict), axis=-1)
#     resultFile.append(testPicture.name + ',' + class_names[res[0]])
    
#     total += 1
#     print("Total images : " + str(total), end='\r')
#     # print(resultFile)
    
    
    
# f = open('test.csv', 'w')
# f.write('filename,label')
# f.write('\n')
# for resultLine in resultFile:
#     f.write(resultLine)
#     f.write('\n')
# f.close()

90000
